In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense,Bidirectional,Dropout
import json
import pickle
import os
from pathlib import Path

In [19]:
def build_path(*path_parts):
    base_dir = Path.cwd().parent
    return os.path.join(base_dir, *path_parts)

In [20]:
X_test = np.load(build_path('Data', 'X_test.npy'))
y_test = np.load(build_path('Data', 'y_test.npy'))
X_train = np.load(build_path('Data', 'X_train.npy'))
y_train = np.load(build_path('Data', 'y_train.npy'))

# Print the shape of the data
print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Testing data shape: {X_test.shape}, Testing labels shape: {y_test.shape}')


Training data shape: (40000, 400), Training labels shape: (40000,)
Testing data shape: (10000, 400), Testing labels shape: (10000,)


In [21]:
vocab_size = 10000
max_len = 400

model = tf.keras.Sequential([
    Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
    Bidirectional(tf.keras.layers.LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [22]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 400, 64)           640000    
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               66048     
 onal)                                                           
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 714369 (2.73 MB)
Trainable params: 714369 (2.73 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [23]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [24]:
## Create an instance of EarlyStoppping Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
earlystopping

In [25]:
## Train the model with early sstopping
history=model.fit(
    X_train,y_train,epochs=10,batch_size=32,
    validation_data=(X_test,y_test),
    validation_split=0.2,
    callbacks=[earlystopping],
)

Epoch 1/10
1250/1250 [==============================] - 134s 106ms/step - loss: 0.4654 - accuracy: 0.7836 - val_loss: 0.4102 - val_accuracy: 0.8120
Epoch 2/10
1250/1250 [==============================] - 149s 119ms/step - loss: 0.3037 - accuracy: 0.8827 - val_loss: 0.3120 - val_accuracy: 0.8816
Epoch 3/10
1250/1250 [==============================] - 149s 120ms/step - loss: 0.2371 - accuracy: 0.9122 - val_loss: 0.3059 - val_accuracy: 0.8799
Epoch 4/10
1250/1250 [==============================] - 120s 96ms/step - loss: 0.2023 - accuracy: 0.9265 - val_loss: 0.3061 - val_accuracy: 0.8825
Epoch 5/10
1250/1250 [==============================] - 130s 104ms/step - loss: 0.1923 - accuracy: 0.9304 - val_loss: 0.4008 - val_accuracy: 0.8536
Epoch 6/10
1250/1250 [==============================] - 120s 96ms/step - loss: 0.1274 - accuracy: 0.9579 - val_loss: 0.4023 - val_accuracy: 0.8763
Epoch 7/10
1250/1250 [==============================] - 120s 96ms/step - loss: 0.1572 - accuracy: 0.9412 - val_los

In [26]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")


313/313 [==============================] - 7s 21ms/step - loss: 0.3059 - accuracy: 0.8799
Test Accuracy: 0.8799


In [27]:
## Save model file
model.save(build_path('Model', 'Model_scratch.h5'))

/mnt/utsav-raj/Project/IMDB/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
